In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

print(tf.test.is_gpu_available())

train_df = pd.read_csv('task3_2021.training.csv')
test_df = pd.read_csv('task3_2021.test.csv')
n_pre_train_y = train_df['target']
n_pre_train_x = train_df.drop(['target'], axis=1)
n_pre_test_x = test_df
# display(n_pre_train_x.head(), n_pre_train_y.head(), n_pre_train_y.nunique(), n_pre_test_x.head())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2021-12-21 03:06:10.074229: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-21 03:06:11.156848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 03:06:11.157336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 03:06:11.161435: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

True
time: 16.6 s (started: 2021-12-21 03:05:58 +00:00)


In [2]:
def oh_pre(x):
    for k in ['f15', 'f16', 'f17']:
        tmp_one_hot_k =  pd.get_dummies(x[k], prefix=f"{k}")
        x = pd.concat([x, tmp_one_hot_k],axis=1)
        x = x.drop([k], axis=1)
    return x

oh_pre_train_x = oh_pre(n_pre_train_x)
oh_pre_test_x = oh_pre(n_pre_test_x)
scaler = StandardScaler()
train_x = scaler.fit_transform(oh_pre_train_x)
test_x = scaler.transform(oh_pre_test_x)

ohenc = OneHotEncoder(handle_unknown="ignore", sparse=False)
train_y = ohenc.fit_transform(n_pre_train_y.values.reshape(-1,1))
# mse = make_scorer(mean_squared_error, squared=False)nunique

time: 126 ms (started: 2021-12-21 03:06:15 +00:00)


In [8]:
class DNN(tf.keras.Model):
  def __init__(self):
    super(DNN,self).__init__()
    self.dense1 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    # x = self.dense2(x)
    # x = self.dense3(x)
    return self.output_layer(x)

def scheduler(epoch, lr):
    if epoch < 100:
        return lr
    else:
        return lr*tf.math.exp(-1e-1)


callback_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

batch_size = 100
epochs = 1000
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model = DNN()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback_lr,callback_early], verbose=1)
model.summary()

Epoch 1/1000
90/90 [==============================] - 0s 3ms/step - loss: 2.2217 - accuracy: 0.1891 - val_loss: 1.9724 - val_accuracy: 0.2930 - lr: 0.0010
Epoch 2/1000
90/90 [==============================] - 0s 2ms/step - loss: 1.8156 - accuracy: 0.3121 - val_loss: 1.7059 - val_accuracy: 0.3560 - lr: 0.0010
Epoch 3/1000
90/90 [==============================] - 0s 2ms/step - loss: 1.6049 - accuracy: 0.3718 - val_loss: 1.5341 - val_accuracy: 0.3970 - lr: 0.0010
Epoch 4/1000
90/90 [==============================] - 0s 2ms/step - loss: 1.4545 - accuracy: 0.4254 - val_loss: 1.4033 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 5/1000
90/90 [==============================] - 0s 2ms/step - loss: 1.3347 - accuracy: 0.4737 - val_loss: 1.2976 - val_accuracy: 0.4910 - lr: 0.0010
Epoch 6/1000
90/90 [==============================] - 0s 2ms/step - loss: 1.2337 - accuracy: 0.5194 - val_loss: 1.2065 - val_accuracy: 0.5390 - lr: 0.0010
Epoch 7/1000
90/90 [==============================] - 0s 2ms/step - lo

In [10]:
from pandas import RangeIndex

nn_pred = model.predict(test_x)
pred = np.where(nn_pred<0.5, 0, 1)
output = ohenc.inverse_transform(pred)
check = (output==None).sum()


if check==0:
    pd.Series(output.reshape(-1), index=RangeIndex(1, 2001), name='Predicted').to_csv('ans_nn_2.csv',index_label='id')

time: 76.7 ms (started: 2021-12-21 03:29:41 +00:00)
